In [1]:
## Basic imports 
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'gender_images/'

os.chdir(base_dir)
os.listdir()

path_training = 'Training/'
path_validation = 'Validation/'

pic_path = path_training+'female/131435.jpg.jpg'
image_size = (90, 90)
batch_size = 128

target_size = (224, 224)
input_shape=(224, 224, 3)
min_lr = 0.0001
steps_per_epoch=256
validation_steps=256
epochs=8


train_datagen = ImageDataGenerator(rescale = 1./255,
      rotation_range=25,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')








test_datagen = ImageDataGenerator( rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
    path_training,
    validation_split = 0.2,
    seed=10,
    target_size=(98,76),
    batch_size=batch_size,
    classes=['female','male'],
    shuffle=True
)


validation_generator = test_datagen.flow_from_directory(
    path_validation,
    seed=10,
    target_size=(98,76),
    batch_size=batch_size,
    classes=['female','male'],
    shuffle=True
)




from matplotlib.image import imread

takeapeak = imread(pic_path)
takeapeak.shape







Found 47035 images belonging to 2 classes.
Found 11643 images belonging to 2 classes.


(98, 76, 3)

In [ ]:

##print("Image size of train ",train_generator[0].shape)

In [3]:
from keras.optimizers.optimizer_v2.rmsprop import RMSProp
## Build the VGG model 

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Flatten, Dense
from keras.applications import VGG16


vgg=keras.applications.VGG16(include_top=False, pooling='avg', weights='imagenet',
input_shape=(98,76, 3))


# Freeze the layers except the last 5
for layer in vgg.layers[:-5]:
 layer.trainable = False# Check the trainable status of the individual layers
for layer in vgg.layers:
 print(layer, layer.trainable)

 # Create the model
model = Sequential()# Add the VGG16 convolutional base model
model.add(vgg)
 
# Add new layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(2, activation='sigmoid'))

## Compile and Summary
##model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=RMSProp(lr=0.1), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

58889256/58889256 [==============================] - 2s 0us/step
<keras.engine.input_layer.InputLayer object at 0x7f7d1767e410> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d85860790> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d85872290> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f7d87ab1d50> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14cd44d0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14ce24d0> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f7d14ce2750> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14de0ad0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14cc6350> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14ce5510> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f7d14ce5110> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f7d14cd4410> False
<keras.layers.convolutional.con

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [6]:



model_with_tuning = 'best_tune_model.h5'
checkpoint_callback = ModelCheckpoint(model_with_tuning,
                                     monitor='accuracy',
                                      verbose=2,
                                     save_best_only=True,mode='max')

history_with_tuning = model.fit(train_generator,
                   steps_per_epoch=len(train_generator) // 128,
                   epochs=3,
                   validation_data=validation_generator,
                   validation_steps=len(validation_generator)// 128,
                   verbose=1,
                   callbacks=[checkpoint_callback])





Epoch 1/3
2/2 [==============================] - ETA: 0s - loss: 0.1797 - accuracy: 0.9258 
Epoch 1: accuracy improved from -inf to 0.92578, saving model to best_tune_model.h5
2/2 [==============================] - 132s 93s/step - loss: 0.1797 - accuracy: 0.9258
Epoch 2/3
2/2 [==============================] - ETA: 0s - loss: 0.2128 - accuracy: 0.9258 
Epoch 2: accuracy did not improve from 0.92578
2/2 [==============================] - 64s 24s/step - loss: 0.2128 - accuracy: 0.9258
Epoch 3/3
2/2 [==============================] - ETA: 0s - loss: 0.1659 - accuracy: 0.9258 
Epoch 3: accuracy did not improve from 0.92578
2/2 [==============================] - 73s 32s/step - loss: 0.1659 - accuracy: 0.9258


In [5]:

# obtain predicted activation values for the last dense layer
path_test = 'Test/'
test1_datagen = ImageDataGenerator( rescale = 1.0/255)
test_generator = test1_datagen.flow_from_directory(
    path_test,
    seed=10,
    target_size=(98,76),
    batch_size=batch_size,
    classes=['female','male'],
    shuffle=True
)

pred = model.predict_generator(test_generator, verbose=1, steps=1000)# determine the maximum activation value for each sample
predicted_class_indices=np.argmax(pred,axis=1)



Found 9 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  del sys.path[0]


   1/1000 [..............................] - ETA: 24:00

1000/1000 [==============================] - 2s 919us/step
